In [1]:
pip install psycopg2-binary

  Using cached psycopg2_binary-2.9.10-cp311-cp311-macosx_14_0_arm64.whl.metadata (4.9 kB)
Using cached psycopg2_binary-2.9.10-cp311-cp311-macosx_14_0_arm64.whl (3.3 MB)
Note: you may need to restart the kernel to use updated packages.


<h1>환경변수 설정</h1>

In [ ]:
from dotenv import load_dotenv
import os
import psycopg2

load_dotenv()

POSTGRE_DB_NAME = os.getenv("POSTGRE_DB_NAME")
POSTGRE_DB_USER = os.getenv("POSTGRE_DB_USER")
POSTGRE_DB_HOST = os.getenv("POSTGRE_DB_HOST")
POSTGRE_DB_PORT = os.getenv("POSTGRE_DB_PORT")
POSTGRE_DB_PASSWORD = os.getenv("POSTGRE_DB_PASSWORD")

# env value check
print(POSTGRE_DB_PORT)

5432


In [ ]:
conn = psycopg2.connect(
        dbname=POSTGRE_DB_NAME,
        user=POSTGRE_DB_USER,
        host=POSTGRE_DB_HOST,
        port=POSTGRE_DB_PORT,
        password=POSTGRE_DB_PASSWORD
    )


try:
    # 비밀번호 없이 연결
    with conn.cursor() as cursor:        
        cursor = conn.cursor()
        cursor.execute("SELECT * FROM documents;")
        print(cursor.fetchall())

except Exception as e:
    print(f"Error: {e}")

finally:
    conn.close()
    print('db 연결 종료!')

<h1></h1>